# Pyspark Basics Practise Notebook

In [1]:
import pyspark
from pyspark import SparkContext,SparkConf
conf=SparkConf().setAppName('RDD_practise').setMaster("local[*]")
sc=SparkContext(conf=conf)
print(sc)

23/01/05 01:47:56 WARN Utils: Your hostname, zourner-HP-Laptop-15s-du1xxx resolves to a loopback address: 127.0.1.1; using 10.41.166.28 instead (on interface wlo1)
23/01/05 01:47:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/05 01:48:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
<SparkContext master=local[*] appName=RDD_practise>


In [2]:
import random
randlist=random.sample(range(0,40),10)
randlist

[8, 9, 32, 26, 39, 27, 17, 31, 22, 18]

In [3]:
# creating RDD
rdd1=sc.parallelize(randlist,4) # if we do not specify number of partition spark will set it to 2
rdd1.collect() # it will print out all the data together

[8, 9, 32, 26, 39, 27, 17, 31, 22, 18]

In [4]:
#data distribution in Partitions
print(rdd1.getNumPartitions())
print(rdd1.glom().collect()) #it will show you where or in which partition data lies
print(rdd1.glom().take(2)) # take all data show it in partitions and only show two partitions

4


[[8, 9], [32, 26], [39, 27], [17, 31, 22, 18]]
[[8, 9], [32, 26]]


In [5]:
# count the number of elements
rdd1.count()

10

In [6]:
# return the three largest values in the data after sorting them in ascending order
rdd1.top(3)

[39, 32, 31]

In [7]:
#distinct()
rdd1.distinct().collect()

[8, 32, 9, 17, 26, 22, 18, 39, 27, 31]

In [8]:
#map(): maps each element with the function passed and retruns the new transformed RDD
rdd2=rdd1.map(lambda c: c/100)
rdd2.collect()

[0.08, 0.09, 0.32, 0.26, 0.39, 0.27, 0.17, 0.31, 0.22, 0.18]

In [9]:
rdd2.getNumPartitions()

4

In [10]:
rdd2.glom().collect() # so we can observe that it gives rdd which is exactly of same number of partitions.

[[0.08, 0.09], [0.32, 0.26], [0.39, 0.27], [0.17, 0.31, 0.22, 0.18]]

In [11]:
rdd3=rdd2.filter(lambda x:x>0.19)#filters the rdd

In [12]:
rdd3.glom().collect()

[[], [0.32, 0.26], [0.39, 0.27], [0.31, 0.22]]

In [13]:
#repartion the small size data into smaller partitions
rdd4=rdd3.repartition(2)

In [14]:
rdd4.glom().collect()

[[0.39, 0.27, 0.31, 0.22], [0.32, 0.26]]

In [17]:
#flatmap()
print('Demonstration through map :',rdd1.map(lambda x:[x+1,x+2]).collect())
print('Demonstration of same using flatmap :',rdd1.flatMap(lambda x:[x+1,x+2]).collect())

Demonstration through map : [[9, 10], [10, 11], [33, 34], [27, 28], [40, 41], [28, 29], [18, 19], [32, 33], [23, 24], [19, 20]]
Demonstration of same using flatmap : [9, 10, 10, 11, 33, 34, 27, 28, 40, 41, 28, 29, 18, 19, 32, 33, 23, 24, 19, 20]


In [19]:
#reduce()
rdd1.reduce(lambda x,y:x+y)

229

In [21]:
# Descriptive Statistics
print('maximum :',rdd1.max())
print('minimum :',rdd1.min())
print('standard deviation :',rdd1.stdev())
print('summation :',rdd1.sum())

maximum : 39
minimum : 8
standard deviation : 9.533624704172071
summation : 229


In [22]:
#mapPartitions() : can be better understood as a function to apply particular function to each partition and showcase results for each partition
def func(partitions):
    sum=0
    for item in partitions:
        sum+=item
    yield sum
    
rdd1.mapPartitions(func).collect()

[17, 58, 66, 88]